In [2]:
import json
import pandas as pd
import gf_utils
from gf_utils.stc_data import get_stc_data
from pathlib import Path
from datetime import datetime
from collections import defaultdict

log_dir = Path('logs/equip')
output_file = Path('equip.csv')
full_record_df = pd.read_csv(output_file)
game_data = get_stc_data(Path(gf_utils.__file__).resolve().parent/'../../data/ch/formatted/json')

In [5]:
from rich.table import Table
from rich.console import Console
from rich.style import NULL_STYLE
from datetime import datetime
td = int(datetime(2022,11,18,10).timestamp())
console = Console(force_terminal=True,force_jupyter=False,no_color=True,highlight=False)
for formula in game_data['recommended_formula'].values():
    # print(formula)
    if formula['develop_type']!=3:
        continue
    mp,ammo,mre,part = formula['mp'],formula['ammo'],formula['mre'],formula['part']
    common_dev_df = full_record_df.query('mp==@mp and ammo==@ammo and mre==@mre and part==@part and item_num==1 and trust==True and dev_time>@td')['equip_id'].value_counts().to_frame('count')
    common_dev_df['chance'] = common_dev_df['count'].apply(lambda x: round(x/common_dev_df['count'].sum()*100,3))
    special_dev_df = full_record_df.query('mp==@mp and ammo==@ammo and mre==@mre and part==@part and item_num==0 and trust==True and dev_time>@td')['equip_id'].value_counts().to_frame('count')
    special_dev_df['chance'] = special_dev_df['count'].apply(lambda x: round(x/special_dev_df['count'].sum()*100,3))
    dev_df = common_dev_df.join(special_dev_df,rsuffix='_special',how='outer').fillna(0).reset_index()
    dev_df['count_special'] = dev_df['count_special'].astype(int)
    for key in ['name','rank']:
        dev_df[key] = dev_df['index'].apply(lambda x: game_data['equip'][x][key])
    footer = f"Count: common {dev_df['count'].sum()}, special {dev_df['count_special'].sum()}\n"
    dev_df = dev_df.query('rank==5').sort_values('chance',ascending=False)
    footer += f"Rank5: common {dev_df['chance'].sum():.3f}, special {dev_df['chance_special'].sum():.3f}"
    table = Table('           name','common','special',header_style=NULL_STYLE, title_style=NULL_STYLE, style=NULL_STYLE, caption=footer, caption_justify='left', caption_style=NULL_STYLE)
    for col in table.columns:
        col.justify='right'
    table.title = f"Formula: {mp} {ammo} {mre} {part}"
    for idx, record in dev_df.iterrows():
        table.add_row(record['name'],*[f'{record[k]:.3f}' for k in ['chance','chance_special']])
    console.print(table)
    console.print()

        Formula: 140 10 110 10        
┌─────────────────┬────────┬─────────┐
│            name │ common │ special │
├─────────────────┼────────┼─────────┤
│     VFL 6-24X56 │  1.984 │   1.606 │
│         EOT 518 │  1.984 │   0.803 │
│        ITI MARS │  0.794 │   0.201 │
│       AC4消音器 │  0.397 │   0.000 │
│      SF武器灯改 │  0.397 │   0.000 │
└─────────────────┴────────┴─────────┘
Count: common 252, special 498        
Rank5: common 5.556, special 2.610    

        Formula: 140 10 110 10        
┌─────────────────┬────────┬─────────┐
│            name │ common │ special │
├─────────────────┼────────┼─────────┤
│     VFL 6-24X56 │  1.984 │   1.606 │
│         EOT 518 │  1.984 │   0.803 │
│        ITI MARS │  0.794 │   0.201 │
│       AC4消音器 │  0.397 │   0.000 │
│      SF武器灯改 │  0.397 │   0.000 │
└─────────────────┴────────┴─────────┘
Count: common 252, special 498        
Rank5: common 5.556, special 2.610    

        Formula: 140 10 110 10        
┌─────────────────┬────────┬─────────

In [3]:
output_file = Path('gun_nm.csv')
full_record_df = pd.read_csv(output_file)
game_data = get_stc_data(Path(gf_utils.__file__).resolve().parent/'../../data/ch/formatted/json')

In [4]:
ts1 = datetime(2022,10,28,10).timestamp()
ts2 = datetime(2022,12,9,10).timestamp()
ts3 = datetime(2022,12,10).timestamp()
ts4 = datetime(2022,12,12).timestamp()
df1 = full_record_df.query('@ts1<=dev_time and dev_time<=@ts2')
df2 = full_record_df.query('@ts4<=dev_time')
df3 = full_record_df.query('@ts3<=dev_time and dev_time<=@ts4')
for i in range(2,6):
    print(df1[f'trust_{i}'].sum())
    print(df2[f'trust_{i}'].sum())
    print(df3[f'trust_{i}'].sum())

86215
20190
39860
35892
8634
28994
17361
4404
23799
4895
1161
7647


In [16]:
cfg = [
    [130,130,130,30],
    [400,400,100,200],
    [400,100,400,200],
    [100,400,400,200]
]
for mp,ammo,mre,part in cfg:
    print(mp,ammo,mre,part)
    df_t2 = df3.query('mp==@mp and ammo==@ammo and mre==@mre and part==@part and trust_2==True')
    len2 = len(df_t2)
    vc2 = df_t2['gun_id'].value_counts(normalize=True)
    df_t5 = df3.query('mp==@mp and ammo==@ammo and mre==@mre and part==@part and trust_5==True')
    len5 = len(df_t5)
    vc5 = df_t5['gun_id'].value_counts(normalize=True)
    print(len2, len5)
    for i in range(375,379):
        if i in vc2:
            print(i, f'{vc5.get(i,0)*100:.2f}%({len5})|{vc2.get(i,0)*100:.3f}%({len2})')

130 130 130 30
8616 1045
375 11.48%(1045)|0.487%(8616)
400 400 100 200
8636 1397
376 8.30%(1397)|0.452%(8636)
378 2.86%(1397)|0.104%(8636)
400 100 400 200
5667 1277
377 7.75%(1277)|0.424%(5667)
378 1.88%(1277)|0.141%(5667)
100 400 400 200
6316 1911
378 6.23%(1911)|0.522%(6316)


## 出率
公式 | 人形 | 五星内出率 | 总出率 | 备注
--|--|--|--|--
1110 | COLT | 11.48%(1045)|0.487%(8616)
4412 | ERMA |8.30%(1397)|0.452%(8636)| 五星内歪SCARL 2.86%
4142 | SCARH | 7.75%(1277)|0.424%(5667) | 五星内歪SCARH 1.88%
1442 | SCARL | 6.23%(1911)|0.522%(6316)

## 出率（UP结束后）
公式 | 人形 | 五星内出率 | 总出率 | 备注
--|--|--|--|--
1110 | COLT | 7.27%(110)|0.272%(2941)
4412 | ERMA |3.85%(156)|0.193%(3112)| 五星内歪SCARL 0.64%
4142 | SCARH | 2.48%(161)|0.109%(2758)| 五星内歪SCARH 0.62%
1442 | SCARL | 2.19%(365)|0.182%(4394)

In [17]:
cfg = [
    [130,130,130,30],
    [400,400,100,200],
    [400,100,400,200],
    [100,400,400,200]
]
for mp,ammo,mre,part in cfg:
    print(mp,ammo,mre,part)
    df_t2 = df2.query('mp==@mp and ammo==@ammo and mre==@mre and part==@part and trust_2==True')
    len2 = len(df_t2)
    vc2 = df_t2['gun_id'].value_counts(normalize=True)
    df_t5 = df2.query('mp==@mp and ammo==@ammo and mre==@mre and part==@part and trust_5==True')
    len5 = len(df_t5)
    vc5 = df_t5['gun_id'].value_counts(normalize=True)
    print(len2, len5)
    for i in range(375,379):
        if i in vc2:
            print(i, f'{vc5.get(i,0)*100:.2f}%({len5})|{vc2.get(i,0)*100:.3f}%({len2})')

130 130 130 30
2941 110
375 7.27%(110)|0.272%(2941)
400 400 100 200
3112 156
376 3.85%(156)|0.193%(3112)
378 0.64%(156)|0.032%(3112)
400 100 400 200
2758 161
377 2.48%(161)|0.109%(2758)
378 0.62%(161)|0.036%(2758)
100 400 400 200
4394 365
378 2.19%(365)|0.182%(4394)


## 出率（UP后）
公式 | 人形 | 五星内出率 | 总出率 | 备注
--|--|--|--|--
1110 | COLT | 7.27%(110)|0.272%(2941)
4412 | ERMA |3.85%(156)|0.193%(3112)| 五星内歪SCARL 0.64%
4142 | SCARH | 2.48%(161)|0.109%(2758)| 五星内歪SCARH 0.62%
1442 | SCARL | 2.19%(365)|0.182%(4394)



In [57]:
vc2 = df_t2['gun_id'].value_counts()
vc2.keys()

Int64Index([ 68,  74,  63,  71, 107,  58,  61,  70, 133,  17,  24,  93,  94,
            105, 288,  31,  92,  27,  33,  21, 349, 262, 330,  32,  60,  69,
             26, 237,  25,  18, 118,  66, 216, 171,  72, 236, 214, 130, 356,
             62,  64, 378, 338, 122, 106,  65,  16, 313, 365, 206, 290,  22,
            243, 215, 205, 306, 196, 172, 287, 181, 194, 129, 293],
           dtype='int64')

In [12]:
datetime.fromtimestamp(df2['dev_time'].min())

datetime.datetime(2022, 12, 12, 0, 0, 2)

In [14]:
df3

,id,dev_type,user_id,build_slot,dev_uname,dev_lv,gun_id,mp,ammo,mre,...,item1_num,core,dev_time,gun_rank,gun_name,file,trust_2,trust_3,trust_4,trust_5
104280,41673278,0,15672,3,黑花,184,349,400,400,400,...,1,0,1670601607,4,AK74M,221210-000657.json,False,False,False,False
104281,41673280,0,108502,3,SHINKAI華,180,95,400,100,400,...,1,0,1670601609,3,汉阳造88式,221210-000657.json,False,False,False,False
104282,41673282,0,15672,1,黑花,184,216,400,400,400,...,1,0,1670601612,4,XM8,221210-000657.json,False,False,False,False
104283,41673286,0,108502,5,SHINKAI華,180,270,400,100,400,...,1,0,1670601613,4,四式,221210-000657.json,False,False,False,False
104284,41673290,0,15672,1,黑花,184,150,400,400,400,...,1,0,1670601617,4,希普卡,221210-000657.json,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206475,41840797,0,102642,1,在下路過的,161,91,130,130,130,...,1,0,1670774394,2,MP-446,221212-000742.json,False,False,False,False
206476,41840798,0,102642,3,在下路過的,161,100,130,130,130,...,1,0,1670774394,4,P7,221212-000742.json,False,True,True,False
206477,41840799,0,102642,5,在下路過的,161,10,130,130,130,...,1,0,1670774394,2,PPK,221212-000742.json,False,False,False,False
206478,41840801,0,760262,1,Bonahc,152,328,400,400,100,...,1,0,1670774397,4,AR-57,221212-000742.json,False,True,True,False
